In [ ]:
import json
import os
import sys
import pandas as pd

sys.path.insert(0, os.path.dirname(os.getcwd()))

In [ ]:
def load_data(path):
    with open(path, "r") as f:
        data = json.load(f)
    return data

In [ ]:
from app.oura_client import HeartRateData, HeartRateSample, _parse_sleep_data
sleep = load_data("./user_data/sleep.json")
sleep = [
    _parse_sleep_data(sleep_record) for sleep_record in sleep['data']
]


In [ ]:
heartrate = load_data("./user_data/heartrate.json")
heartrate = [
                HeartRateSample(
                    bpm=item.get("bpm", 0),
                    source=item.get("source", ""),
                    timestamp=item.get("timestamp", ""),
                )
                for item in heartrate.get("data", [])
            ]
heartrate = HeartRateData(data=heartrate)

In [ ]:
from app.analytics import oura_sleep_to_dataframe, oura_heartrate_to_dataframe

sleep_df = oura_sleep_to_dataframe(sleep)
heartrate_df = oura_heartrate_to_dataframe(heartrate)

In [ ]:
sleep_df.head()

In [ ]:
heartrate_df["source"].value_counts()

In [ ]:
(12000*5)/(60*24)

In [ ]:
from app.analytics import analyze_heart_rate_daily


per_day_heartrate = analyze_heart_rate_daily(heartrate_df)

In [ ]:
per_day_heartrate

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Convert to DataFrame for easier plotting
daily_df = pd.DataFrame([
    {
        "day": d.day,
        "average_hr": d.average_hr,
        "hr_20th": d.hr_20th_percentile,
        "hr_50th": d.hr_50th_percentile,
        "hr_80th": d.hr_80th_percentile,
        "hr_95th": d.hr_95th_percentile,
    }
    for d in per_day_heartrate
])

# 7-day rolling average
daily_df["hr_50th_7d"] = daily_df["hr_50th"].rolling(window=7, center=True).mean()
daily_df["average_hr_7d"] = daily_df["average_hr"].rolling(window=7, center=True).mean()

fig, ax = plt.subplots(figsize=(12, 6))

# Plot percentile bands
ax.fill_between(daily_df["day"], daily_df["hr_20th"], daily_df["hr_80th"], 
                alpha=0.2, label="20th-80th percentile", color="steelblue")

# Plot raw data (lighter)
ax.plot(daily_df["day"], daily_df["hr_50th"], 
        color="steelblue", linewidth=1, alpha=0.3)
ax.plot(daily_df["day"], daily_df["average_hr"], 
        color="coral", linewidth=1, alpha=0.3)

# Plot 7-day rolling average trend lines
ax.plot(daily_df["day"], daily_df["hr_50th_7d"], label="Median (7-day avg)", 
        color="steelblue", linewidth=2.5)
ax.plot(daily_df["day"], daily_df["average_hr_7d"], label="Average (7-day avg)", 
        color="coral", linewidth=2.5, linestyle="--")
ax.plot(daily_df["day"], daily_df["hr_95th"], label="95th percentile", 
        color="gray", linewidth=1, linestyle=":")

ax.set_xlabel("Date")
ax.set_ylabel("Heart Rate (bpm)")
ax.set_title("Daily Heart Rate Analytics (Awake)")
ax.legend()

# Grid with vertical lines for each day
ax.set_xticks(daily_df["day"])

# Horizontal lines for each integer bpm value
y_min = int(daily_df["hr_20th"].min()) - 1
y_max = int(daily_df["hr_95th"].max()) + 1

# Minor grid (every 1 bpm) - thin lines
ax.set_yticks(np.arange(y_min, y_max + 1, 1), minor=True)
ax.grid(True, alpha=0.2, which='minor', axis='y')

# Major grid (every 5 bpm) - thick lines
y_min_5 = (y_min // 5) * 5
y_max_5 = ((y_max // 5) + 1) * 5
ax.set_yticks(np.arange(y_min_5, y_max_5 + 1, 5), minor=False)
ax.grid(True, alpha=0.5, which='major', linewidth=1.5)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()